# Clase Práctica #14 (Compilación)

En esta clase implementaremos las primeras fases de chequeo semántico para el lenguaje que comenzamos a estudiar en la clase anterior. Pasemos a importar lo que ya habíamos implementado.

In [14]:
import cmp.nbpackage
import cmp.visitor as visitor

from cp13 import G, text
from cp13 import Node, ProgramNode, DeclarationNode, ExpressionNode
from cp13 import ClassDeclarationNode, FuncDeclarationNode, AttrDeclarationNode
from cp13 import VarDeclarationNode, AssignNode, CallNode
from cp13 import AtomicNode, BinaryNode
from cp13 import ConstantNumNode, VariableNode, InstantiateNode, PlusNode, MinusNode, StarNode, DivNode
from cp13 import FormatVisitor, tokenize_text, pprint_tokens

from cmp.tools.parsing import LR1Parser
from cmp.evaluation import evaluate_reverse_parse

La gramática `G` es la gramática que diseñamos para el lenguaje. Esta debe atrapar la sintaxis del lenguaje. Por otro lado, las reglas que incluimos al atributar la gramática deberían construir una representación sobre la que fuera cómodo comprobar la semántica del programa. Estamos hablando justamente del **AST**.

Construyamos el siguiente método `run_pipeline`, el cual recibe una cadena de texto y una gramática, y pasará por las fases de análisis lexicográfico y sintácticto, y finalmente evaluará las reglas para devolvernos el AST.

In [15]:
def run_pipeline(G, text):
    print('=================== TEXT ======================')
    print(text)
    print('================== TOKENS =====================')
    tokens = tokenize_text(text)
    pprint_tokens(tokens)
    print('=================== PARSE =====================')
    parser = LR1Parser(G)
    parse, operations = parser([t.token_type for t in tokens], get_shift_reduce=True)
    print('\n'.join(repr(x) for x in parse))
    print('==================== AST ======================')
    ast = evaluate_reverse_parse(parse, operations, tokens)
    formatter = FormatVisitor()
    tree = formatter.visit(ast)
    print(tree)
    return ast
    
ast = run_pipeline(G, text)

=================== TEXT ======================

class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b ;
    }
    b : int ;
}

class B : A {
    c : A ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        c ;
    }
}

================== TOKENS =====================
class id {
    id : id ;
    def id ( id : id , id : id ) : id {
        id + id ;
    }
    id : id ;
}
class id : id {
    id : id ;
    def id ( id : id , id : id ) : id {
        let id : id = int ;
        let id = new id ( ) . id ( int , id ) ;
        id ;
    }
}
$
=================== PARSE =====================
<def-attr> -> id : id ;
<param> -> id : id
<param> -> id : id
<param-list> -> <param>
<param-list> -> <param> , <param-list>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <term>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <arith> + <term>
<expr> -> <arith>
<expr-list> -> <expr> ;
<de

## Chequeo semántico

En `cmp.semantic` se distribuyen una serie de clases que funcionarán como soporte para la fase de chequeo semántico.

In [16]:
from cmp.semantic import SemanticError
from cmp.semantic import Attribute, Method, Type
from cmp.semantic import VoidType, ErrorType
from cmp.semantic import Context

- La clase `SemanticError` hereda de `Exception` para funcionar como mecanismo para manejar errores en los contextos. El campo `text` que poseen las instancias de `SemanticError` permite obtener el texto de error con el que se construyó.
- Las clases `Attribute` y `Method` funcionan como contenedores de los datos necesarios para representar los atributos y métodos del lenguaje respectivamente. Del primero se almacena el nombre del campo (un `str)` y su tipo (una instancia de `Type`). Del segundo se almacenan: nombre del método (`str`), nombre de los parámetros (`list<str>`), tipos de los parámetros (`list<Type>`) y el tipo de retorno (`Type`).
- La clase `Type` funciona como descriptor de todos los atributos y métodos con que cuentan los tipos del lenguaje. Esta clase permite crear instancias a partir del nombre del tipo (`Type(name)`) y posteriormente actualizar su definición con:
    - tipo padre: `set_parent(...)`.
    - atributos: `get_attributes(...)` y `define_attribute(...)`.
    - métodos: `get_method(...)` y `define_method(...)`.

> Para más información se recomienda revisar el código fuente disponible en `cmp.semantic`.

- La clase `VoidType` puede usarse para manejar el tipo de retorno `void` de los métodos. Tiene la particularidad de que todas sus instancias son iguales entre sí.
- La clase `ErrorType` puede usarse para manejar las situaciones en las que se refiere un tipo que no ha sido declarado. Esto nos permitirá detectar más errores que detener el chequeo semántico al primer error. Las instancias de `ErrorType` tiene la particularidad de ser iguales entre sí y a cualquier instancia de `Type`. Además, el tipo `ErrorType` se conforma (en el sentido de herencia) a todo tipo y viceversa.
- La clase `Context` permite controlar los tipos que han sido definidos en el lenguaje.
    - definir un tipo: `create_type(...)`.
    - obtener un tipo: `get_type(...)`.

## Recolectando de tipos

Dado que en este lenguaje los tipos pueden referenciarse antes de declararse, se vuelve necesario realizar un primer recorrido del AST recolectando todos los tipos. Esto lo haremos utilizando el patrón `visitor` con el que trabajamos en clases anteriores.

In [17]:
class TypeCollector(object):
    def __init__(self, errors=[]):
        self.context = None
        self.errors = errors
        self.type_level = {}
    
    @visitor.on('node')
    def visit(self, node):
        pass
    
    @visitor.when(ProgramNode)
    def visit(self, node):
        self.context = Context()
        # Your code here!!!
        self.context.create_type('int') # ask for missing built-in types
        
        for def_class in node.declarations:
            self.visit(def_class)
             
        # comparison for sort node.declarations
        def get_type_level(typex):
            try:
                parent = self.type_level[typex]
            except KeyError:
                return 0
            
            if parent == 0:
                self.errors.append('Cyclic heritage.')
            elif type(parent) is not int:
                self.type_level[typex] = 0 if parent else 1
                if type(parent) is str:
                    self.type_level[typex] = get_type_level(parent) + 1
                
            return self.type_level[typex]
        
        node.declarations.sort(key = lambda node: get_type_level(node.id))
                
                
        
    # Your code here!!!
    # ????
    @visitor.when(ClassDeclarationNode)
    def visit(self, node):
        try:
            self.context.create_type(node.id)
            self.type_level[node.id] = node.parent
        except SemanticError as ex:
            self.errors.append(ex.text)


Comprobemos que implementamos correctamente el recorrido. Tras visitar el AST deberíamos tener en el campo `context` del `visitor` todos los tipos definidos en el programa _... y algo más? ;-)_

In [18]:
errors = []

collector = TypeCollector(errors)
collector.visit(ast)

context = collector.context

print('Errors:', errors)
print('Context:')
print(context)

assert errors == []

Errors: []
Context:
{
	type int {}
	
	type A {}
	
	type B {}
	
}


## Construyendo los tipos

Pasemos ahora a construir los tipos. Pero _... realmente podremos comenzar ya a chequear todo el código? (incluyendo el cuerpo de los métodos)_. Resulta que no. En este lenguaje en orden en que se definen los métodos tampoco es relevante: _se pueden llamar antes de declararse_. Esto permite que haya recursividad en el lenguaje que lleva un chequeo extra antes de pasar a revisar los cuerpos de los métodos.

Nótese que al haber recolectado ya todos los tipos, se logra que los parámetros, valores de retorno, y otras refencias a tipos, puedan ser resueltas en este recorrido sin problemas.

In [19]:
class TypeBuilder:
    def __init__(self, context, errors=[]):
        self.context = context
        self.current_type = None
        self.errors = errors
    
    @visitor.on('node')
    def visit(self, node):
        pass
    
    # Your code here!!!
    # ????
    @visitor.when(ProgramNode)
    def visit(self, node):
        for def_class in node.declarations:
            self.visit(def_class)
            
        try:
            self.context.get_type('Main').get_method('main')
        except SemanticError:
            self.errors.append('The class "Main" and his method "main" are needed.')
            
    
    @visitor.when(ClassDeclarationNode)
    def visit(self, node):
        self.current_type = self.context.get_type(node.id)
        
        if node.parent:
            try:
                parent_type = self.context.get_type(node.parent)
                self.current_type.set_parent(parent_type)
            except SemanticError as ex:
                self.errors.append(ex.text)
        
        for feature in node.features:
            self.visit(feature)
            
    @visitor.when(AttrDeclarationNode)
    def visit(self, node):
        try:
            attr_type = self.context.get_type(node.type)
        except SemanticError as ex:
            self.errors.append(ex.text)
            attr_type = ErrorType()
            
        try:
            self.current_type.define_attribute(node.id, attr_type)
        except SemanticError as ex:
            self.errors.append(ex.text)
        
    @visitor.when(FuncDeclarationNode)
    def visit(self, node):
        arg_names, arg_types = [], []
        for idx, typex in node.params:
            try:
                arg_type = self.context.get_type(typex)
            except SemanticError as ex:
                self.errors.append(ex.text)
                arg_type = ErrorType()
                
            arg_names.append(idx)
            arg_types.append(arg_type)
        
        try:
            ret_type = VoidType() if node.type == 'void' else self.context.get_type(node.type)
        except SemanticError as ex:
            self.errors.append(ex.text)
            ret_type = ErrorType()
        
        try:
            self.current_type.define_method(node.id, arg_names, arg_types, ret_type)
        except SemanticError as ex:
            self.errors.append(ex.text)
        


Comprobemos la implementación. Tras esta fase deberíamos tener completados todos los tipos.

In [20]:
builder = TypeBuilder(context, errors)
builder.visit(ast)

print('Errors:', errors)
print('Context:')
print(context)

Errors: ['The class "Main" and his method "main" are needed.']
Context:
{
	type int {}
	
	type A {
		[attrib] a : int;
		[attrib] b : int;
		[method] suma(a:int, b:int): int;
	}
	
	type B : A {
		[attrib] c : A;
		[method] f(d:int, a:A): <void>;
	}
	
}


## Comprobando ...

In [21]:
deprecated_pipeline = run_pipeline

Actualizaremos el pipeline para incluir estos 2 recorridos.

In [22]:
def run_pipeline(G, text):
    ast = deprecated_pipeline(G, text)
    print('============== COLLECTING TYPES ===============')
    errors = []
    collector = TypeCollector(errors)
    collector.visit(ast)
    context = collector.context
    print('Errors:', errors)
    print('Context:')
    print(context)
    print('=============== BUILDING TYPES ================')
    builder = TypeBuilder(context, errors)
    builder.visit(ast)
    print('Errors: [')
    for error in errors:
        print('\t', error)
    print(']')
    print('Context:')
    print(context)
    return ast, errors, context

### Programa #1

El siguiente programa es con el que hemos estado trabajando y no debería contener errores.

In [23]:
from cp13 import text

ast, errors, context = run_pipeline(G, text)
assert errors == ['The class "Main" and his method "main" are needed.']

=================== TEXT ======================

class A {
    a : int ;
    def suma ( a : int , b : int ) : int {
        a + b ;
    }
    b : int ;
}

class B : A {
    c : A ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        c ;
    }
}

================== TOKENS =====================
class id {
    id : id ;
    def id ( id : id , id : id ) : id {
        id + id ;
    }
    id : id ;
}
class id : id {
    id : id ;
    def id ( id : id , id : id ) : id {
        let id : id = int ;
        let id = new id ( ) . id ( int , id ) ;
        id ;
    }
}
$
=================== PARSE =====================
<def-attr> -> id : id ;
<param> -> id : id
<param> -> id : id
<param-list> -> <param>
<param-list> -> <param> , <param-list>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <term>
<atom> -> id
<factor> -> <atom>
<term> -> <factor>
<arith> -> <arith> + <term>
<expr> -> <arith>
<expr-list> -> <expr> ;
<de

### Programa #2

Se incluyeron varios errores al programa anterior. Intente detectar todos los errores posibles.

In [24]:
text = '''
class A {
    a : Z ;
    def suma ( a : int , b : B ) : int {
        a + b ;
    }
    b : int ;
    c : C ;
}

class B : A {
    c : A ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        c ;
    }
    z : int ;
    z : A ;
}

class C : Z {
}

class D : A {
    def suma ( a : int , d : B ) : int {
        d ;
    }
}

class E : A {
    def suma ( a : A , b : B ) : C {
        a ;
    }
}

class F : B {
    def f ( d : int , a : A ) : void {
        a ;
    }
}
'''

ast, errors, context = run_pipeline(G, text)

assert sorted(errors) == sorted([
	 'Type "Z" is not defined.',
	 'Attribute "c" is already defined in B.',
	 'Attribute "z" is already defined in B.',
	 'Type "Z" is not defined.',
	 'Method "suma" already defined in E with a different signature.',
	 'The class "Main" and his method "main" are needed.'
])

=================== TEXT ======================

class A {
    a : Z ;
    def suma ( a : int , b : B ) : int {
        a + b ;
    }
    b : int ;
    c : C ;
}

class B : A {
    c : A ;
    def f ( d : int , a : A ) : void {
        let f : int = 8 ;
        let c = new A ( ) . suma ( 5 , f ) ;
        c ;
    }
    z : int ;
    z : A ;
}

class C : Z {
}

class D : A {
    def suma ( a : int , d : B ) : int {
        d ;
    }
}

class E : A {
    def suma ( a : A , b : B ) : C {
        a ;
    }
}

class F : B {
    def f ( d : int , a : A ) : void {
        a ;
    }
}

================== TOKENS =====================
class id {
    id : id ;
    def id ( id : id , id : id ) : id {
        id + id ;
    }
    id : id ;
    id : id ;
}
class id : id {
    id : id ;
    def id ( id : id , id : id ) : id {
        let id : id = int ;
        let id = new id ( ) . id ( int , id ) ;
        id ;
    }
    id : id ;
    id : id ;
}
class id : id {
}
class id : id {
    def id ( id : i

### Programa #3

El siguente programa tiene un problema: la defición de `A` y `B` forma un ciclo.

In [25]:
text = '''
class A : B {
}
class B : A {
}
'''

ast, errors, context = run_pipeline(G, text)
assert len(errors) != 0

=================== TEXT ======================

class A : B {
}
class B : A {
}

================== TOKENS =====================
class id : id {
}
class id : id {
}
$
=================== PARSE =====================
<feature-list> -> e
<def-class> -> class id : id { <feature-list> }
<feature-list> -> e
<def-class> -> class id : id { <feature-list> }
<class-list> -> <def-class>
<class-list> -> <def-class> <class-list>
<program> -> <class-list>
==================== AST ======================
\__ProgramNode [<class> ... <class>]
	\__ClassDeclarationNode: class A : B { <feature> ... <feature> }

	\__ClassDeclarationNode: class B : A { <feature> ... <feature> }

============== COLLECTING TYPES ===============
Errors: ['Cyclic heritage.']
Context:
{
	type int {}
	
	type A {}
	
	type B {}
	
}
=============== BUILDING TYPES ================
Errors: [
	 Cyclic heritage.
	 The class "Main" and his method "main" are needed.
]
Context:
{
	type int {}
	
	type A : B {}
	
	type B : A {}
	
}


## Propuestas

- Compruebe el hecho de que todo programa del lenguaje deba tener una clase `Main` con un método `main`.